In [67]:
from fastai.imports import *
from fastai.structured import *
from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display
from sklearn import metrics
import os
import pandas as pd
import numpy as np

## Read Data

In [79]:
os.chdir("/home/rk9cx/Kaggle/Kobe Shot Selection/")
df = pd.read_csv("data.csv", index_col = False, low_memory=False, parse_dates=["game_date"])
test = df[df['shot_made_flag'].isna()]
train = df[~df['shot_made_flag'].isna()]

## Preprocessing

In [80]:
#converting date into different fatures
add_datepart(train, 'game_date')
add_datepart(test, 'game_date')

/home/rk9cx/.local/lib/python3.6/site-packages/fastai/structured.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
/home/rk9cx/.local/lib/python3.6/site-packages/fastai/structured.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
/apps/software/standard/core/anaconda/5.2.0-py3.6/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

In [81]:
#converting categorical variables into label coding
train_cats(train)
apply_cats(test, train)

/home/rk9cx/.local/lib/python3.6/site-packages/fastai/structured.py:152: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if is_string_dtype(c): df[n] = c.astype('category').cat.as_ordered()
/home/rk9cx/.local/lib/python3.6/site-packages/fastai/structured.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[n] = pd.Categorical(c, categories=trn[n].cat.categories, ordered=True)


In [82]:
#test.drop(["shot_made_flag"], axis = 1, inplace= True)
df_test, y_test, nas = proc_df(test, 'shot_made_flag')

/apps/software/standard/core/anaconda/5.2.0-py3.6/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [72]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

In [73]:
#checking missing values
display_all(train.isnull().sum().sort_index()/len(train))
#clearly no missing values

action_type              0.0
combined_shot_type       0.0
game_Day                 0.0
game_Dayofweek           0.0
game_Dayofyear           0.0
game_Elapsed             0.0
game_Is_month_end        0.0
game_Is_month_start      0.0
game_Is_quarter_end      0.0
game_Is_quarter_start    0.0
game_Is_year_end         0.0
game_Is_year_start       0.0
game_Month               0.0
game_Week                0.0
game_Year                0.0
game_event_id            0.0
game_id                  0.0
lat                      0.0
loc_x                    0.0
loc_y                    0.0
lon                      0.0
matchup                  0.0
minutes_remaining        0.0
opponent                 0.0
period                   0.0
playoffs                 0.0
season                   0.0
seconds_remaining        0.0
shot_distance            0.0
shot_id                  0.0
shot_made_flag           0.0
shot_type                0.0
shot_zone_area           0.0
shot_zone_basic          0.0
shot_zone_rang

In [83]:
#imputing missing values with median
df, y, nas = proc_df(train, 'shot_made_flag')

/apps/software/standard/core/anaconda/5.2.0-py3.6/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


#using just the top 5 features based on importance
df = df.loc[:,['action_type','combined_shot_type','shot_distance', 'matchup','shot_id']]
df.head()

df_test = df_test.loc[:,['action_type','combined_shot_type','shot_distance', 'matchup','shot_id']]

In [84]:
#function for train-test split
def split_vals(a,n): 
    return a[:n].copy(), a[n:].copy()

n_valid = 5000  # same as Kaggle's test set size
n_trn = len(df)-n_valid
raw_train, raw_valid = split_vals(train, n_trn)
X_train, X_valid = split_vals(df, n_trn)
y_train, y_valid = split_vals(y, n_trn)

## Modelling - Basic Random Forest

In [85]:
#cross validation
m = RandomForestClassifier(n_jobs=-1)
m.fit(X_train, y_train)
metrics.log_loss(y_valid,m.predict_proba(X_valid))

0.7950342674059844

In [50]:
#make submissions
shot_id = df_test["shot_id"].tolist()
submissions = pd.DataFrame({'shot_id': shot_id, 'shot_made_flag': list(m.predict_proba(df_test)[:,1])}, 
                           columns=['shot_id', 'shot_made_flag'])
submissions.to_csv("1.csv", index=False)

## Modelling - Random Forest with More Trees

In [88]:
#cross validation
m = RandomForestClassifier(n_estimators=1000, n_jobs=-1)
m.fit(X_train, y_train)
metrics.log_loss(y_valid,m.predict_proba(X_valid))

0.6326936996997542

In [89]:
#make submissions
shot_id = df_test["shot_id"].tolist()
submissions = pd.DataFrame({'shot_id': shot_id, 'shot_made_flag': list(m.predict_proba(df_test)[:,1])}, 
                           columns=['shot_id', 'shot_made_flag'])
submissions.to_csv("1.csv", index=False)
#looks like we are overfitting with 1000 trees

## Depth Check

In [53]:
#function to calculate maximum depth
def dectree_max_depth(tree):
    children_left = tree.children_left
    children_right = tree.children_right

    def walk(node_id):
        if (children_left[node_id] != children_right[node_id]):
            left_max = 1 + walk(children_left[node_id])
            right_max = 1 + walk(children_right[node_id])
            return max(left_max, right_max)
        else: # leaf
            return 1

    root_node_id = 0
    return walk(root_node_id)

In [54]:
t=m.estimators_[0].tree_
dectree_max_depth(t)

38

## Modelling - Random Forest with Hyperparametric Tuning

In [ ]:
#cross validation
m = RandomForestClassifier(n_estimators=4000, min_samples_leaf=100, max_features=0.5, n_jobs=-1)
m.fit(X_train, y_train)
metrics.log_loss(y_valid,m.predict_proba(X_valid))

In [91]:
import pandas as pd
feature_importances = pd.DataFrame(m.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)
feature_importances

,importance
action_type,0.524405
combined_shot_type,0.106154
shot_distance,0.074735
matchup,0.032132
game_Elapsed,0.025524
game_event_id,0.021305
loc_y,0.020420
lat,0.019955
shot_id,0.019892
seconds_remaining,0.017197


In [92]:
#make submissions
shot_id = df_test["shot_id"].tolist()
submissions = pd.DataFrame({'shot_id': shot_id, 'shot_made_flag': list(m.predict_proba(df_test)[:,1])}, 
                           columns=['shot_id', 'shot_made_flag'])
submissions.to_csv("1.csv", index=False)
#looks like we are overfitting with 1000 trees

## Exploratory Data Analysis

In [36]:
df.transpose()

,1,2,3,4,5,6,8,9,10,11,...,30685,30687,30688,30689,30690,30691,30692,30694,30695,30696
action_type,26,26,26,5,26,27,26,41,26,26,...,26,26,26,26,26,12,26,41,26,26
combined_shot_type,4,4,4,2,4,5,4,4,4,4,...,4,4,4,4,4,5,4,4,4,4
game_event_id,12,35,43,155,244,251,265,294,309,4,...,249,284,308,326,331,382,397,426,448,471
game_id,20000012,20000012,20000012,20000012,20000012,20000012,20000012,20000012,20000012,20000019,...,49900088,49900088,49900088,49900088,49900088,49900088,49900088,49900088,49900088,49900088
lat,34.0443,33.9093,33.8693,34.0443,34.0553,34.0443,33.9363,33.9193,33.8063,33.9173,...,33.7943,33.9443,33.9833,33.3653,33.9443,34.0443,33.9963,33.8783,33.7773,33.9723
loc_x,-157,-101,138,0,-145,0,-65,-33,-94,121,...,81,40,-126,-12,-113,0,1,-134,31,1
loc_y,0,135,175,0,-11,0,108,125,238,127,...,250,100,61,679,100,0,48,166,267,72
lon,-118.427,-118.371,-118.132,-118.27,-118.415,-118.27,-118.335,-118.303,-118.364,-118.149,...,-118.189,-118.23,-118.396,-118.282,-118.383,-118.27,-118.269,-118.404,-118.239,-118.269
minutes_remaining,10,7,6,6,9,8,6,3,1,11,...,7,3,1,0,11,7,6,3,2,0
period,1,1,1,2,3,3,3,3,3,1,...,3,3,3,3,4,4,4,4,4,4


In [30]:
df["action_type"].unique()

array(['Jump Shot', 'Driving Dunk Shot', 'Layup Shot', 'Running Jump Shot', 'Driving Layup Shot',
       'Reverse Layup Shot', 'Reverse Dunk Shot', 'Slam Dunk Shot', 'Turnaround Jump Shot', 'Tip Shot',
       'Running Hook Shot', 'Alley Oop Dunk Shot', 'Dunk Shot', 'Alley Oop Layup shot', 'Running Dunk Shot',
       'Driving Finger Roll Shot', 'Running Layup Shot', 'Finger Roll Shot', 'Fadeaway Jump Shot',
       'Follow Up Dunk Shot', 'Hook Shot', 'Turnaround Hook Shot', 'Running Tip Shot', 'Jump Hook Shot',
       'Running Finger Roll Shot', 'Jump Bank Shot', 'Turnaround Finger Roll Shot', 'Hook Bank Shot',
       'Driving Hook Shot', 'Running Reverse Layup Shot', 'Driving Finger Roll Layup Shot',
       'Fadeaway Bank shot', 'Pullup Jump shot', 'Finger Roll Layup Shot', 'Turnaround Fadeaway shot',
       'Driving Reverse Layup Shot', 'Driving Slam Dunk Shot', 'Step Back Jump shot',
       'Reverse Slam Dunk Shot', 'Turnaround Bank shot', 'Running Finger Roll Layup Shot',
       'Flo

In [31]:
df["combined_shot_type"].unique()

array(['Jump Shot', 'Dunk', 'Layup', 'Tip Shot', 'Hook Shot', 'Bank Shot'], dtype=object)

In [36]:
df[["game_id", "combined_shot_type","action_type"]].groupby(["combined_shot_type", "action_type"]).agg(["count"])
#doesnt look like action_type is adding any extra information in most cases

game_id
                                                        count
combined_shot_type action_type                               
Bank Shot          Driving Bank shot                        5
                   Hook Bank Shot                           5
                   Pullup Bank shot                        12
                   Running Bank shot                       48
                   Turnaround Bank shot                    71
Dunk               Alley Oop Dunk Shot                    122
                   Driving Dunk Shot                      310
                   Driving Slam Dunk Shot                  48
                   Dunk Shot                              262
                   Follow Up Dunk Shot                     15
                   Putback Dunk Shot                        5
                   Putback Slam Dunk Shot                   2
                   Reverse Dunk Shot                       75
                   Reverse Slam Dunk Shot                  16
                   Running Dunk Shot                       19
                   Running Slam Dunk Shot                   1
                   Slam Dunk Shot                         411
Hook Shot          Driving Hook Shot                       14
                   Hook Shot                               84
                   Running Hook Shot                       41
                   Turnaround Hook Shot                    14
Jump Shot          Driving Floating Bank Jump Shot          1
                   Driving Floating Jump Shot               5
                   Driving Jump shot                       28
                   Fadeaway Bank shot                      31
                   Fadeaway Jump Shot                    1048
                   Floating Jump shot                     114
                   Jump Bank Shot                         333
                   Jump Hook Shot                          24
                   Jump Shot                            18880
                   Pullup Jump shot                       476
                   Running Jump Shot                      926
                   Running Pull-Up Jump Shot                4
                   Step Back Jump shot                    118
                   Turnaround Fadeaway Bank Jump Shot       1
                   Turnaround Fadeaway shot               439
                   Turnaround Jump Shot                  1057
Layup              Alley Oop Layup shot                    80
                   Cutting Finger Roll Layup Shot           1
                   Cutting Layup Shot                       6
                   Driving Finger Roll Layup Shot          69
                   Driving Finger Roll Shot                82
                   Driving Layup Shot                    1978
                   Driving Reverse Layup Shot              97
                   Finger Roll Layup Shot                  33
                   Finger Roll Shot                        28
                   Layup Shot                            2567
                   Putback Layup Shot                      15
                   Reverse Layup Shot                     395
                   Running Finger Roll Layup Shot           6
                   Running Finger Roll Shot                 4
                   Running Layup Shot                      72
                   Running Reverse Layup Shot              11
                   Tip Layup Shot                           2
                   Turnaround Finger Roll Shot              2
Tip Shot           Running Tip Shot                         2
                   Tip Shot                               182

In [39]:
df["shot_id"].unique()

array([    1,     2,     3, ..., 30695, 30696, 30697])

In [ ]:
#Next steps
#calculate percentage of success for each type of action type and combined shot type
#calculate 
